In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("/content/DataPreprocessingGraded_dataset.csv")

In [5]:
df.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [6]:
df['V5'].unique()

array(['NEGATIVE'], dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      748 non-null    object 
 1   V2      748 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB


In [8]:
df1 = df.replace('?' , np.nan)

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      743 non-null    object 
 1   V2      743 non-null    object 
 2   V3      748 non-null    float64
 3   V4      748 non-null    float64
 4   V5      748 non-null    object 
 5   Target  748 non-null    object 
dtypes: float64(2), object(4)
memory usage: 35.2+ KB


In [10]:
df_copy = df1.copy()

In [11]:
data = np.array(df_copy)

In [12]:
type(data)

numpy.ndarray

In [13]:
type(df_copy)

pandas.core.frame.DataFrame

In [14]:
data[:5]

array([['2.0', '50.0', 12500.0, 98.0, 'NEGATIVE', 'YES'],
       ['0.0', '13.0', 3250.0, 28.0, 'NEGATIVE', 'YES'],
       [nan, nan, 4000.0, 35.0, 'NEGATIVE', 'YES'],
       [nan, '20.0', 5000.0, 45.0, 'NEGATIVE', 'YES'],
       ['1.0', '24.0', 6000.0, 77.0, 'NEGATIVE', 'NO']], dtype=object)

In [15]:
df1.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'Target'], dtype='object')

In [16]:
X = df1.drop('Target', axis = 1)

In [17]:
X.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5'], dtype='object')

In [18]:
y = df1['Target']

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer

In [20]:
tf = [("ordinal_encode", OrdinalEncoder(), [4])]

num_pipeline = Pipeline([("ct1", ColumnTransformer(transformers = [("select_4", "passthrough", [0,1,2,3])])),
                         ("ct2", ColumnTransformer(transformers = [("impute", SimpleImputer(strategy= "mean"), [0,1])], remainder = "passthrough")),
                         ("ct3", ColumnTransformer(transformers = [("scaler", StandardScaler(), [0,1,2,3])]))])

In [21]:
cat_pipeline = Pipeline([("cat", ColumnTransformer(transformers = tf))])

In [22]:
pre_processed = FeatureUnion([("num_pipeline", num_pipeline),
                              ("cat_pipeline", cat_pipeline)])

In [23]:
pre_processed

FeatureUnion(transformer_list=[('num_pipeline',
                                Pipeline(steps=[('ct1',
                                                 ColumnTransformer(transformers=[('select_4',
                                                                                  'passthrough',
                                                                                  [0,
                                                                                   1,
                                                                                   2,
                                                                                   3])])),
                                                ('ct2',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('impute',
                                                                                  SimpleImputer(),
                                                                                  [0,
                                                                                   1])])),
                                                ('ct3',
                                                 ColumnTransformer(transformers=[('scaler',
                                                                                  StandardScaler(),
                                                                                  [0,
                                                                                   1,
                                                                                   2,
                                                                                   3])]))])),
                               ('cat_pipeline',
                                Pipeline(steps=[('cat',
                                                 ColumnTransformer(transformers=[('ordinal_encode',
                                                                                  OrdinalEncoder(),
                                                                                  [4])]))]))])

In [46]:
X_use = pre_processed.fit_transform(X)

In [48]:
X_use[:5]

array([[-9.38169390e-01,  7.70986653e+00,  7.62334626e+00,
         2.61563344e+00,  0.00000000e+00],
       [-1.18627754e+00,  1.30454949e+00,  1.28273826e+00,
        -2.57880900e-01,  0.00000000e+00],
       [ 0.00000000e+00, -1.53758496e-16,  1.79684161e+00,
         2.94705348e-02,  0.00000000e+00],
       [ 0.00000000e+00,  2.51636623e+00,  2.48231275e+00,
         4.39972584e-01,  0.00000000e+00],
       [-1.06222347e+00,  3.20883294e+00,  3.16778388e+00,
         1.75357914e+00,  0.00000000e+00]])

In [24]:
full_pipeline = Pipeline([("preprocess", pre_processed),
                          ('selector', VarianceThreshold(0.1))])

In [25]:
full_pipeline

Pipeline(steps=[('preprocess',
                 FeatureUnion(transformer_list=[('num_pipeline',
                                                 Pipeline(steps=[('ct1',
                                                                  ColumnTransformer(transformers=[('select_4',
                                                                                                   'passthrough',
                                                                                                   [0,
                                                                                                    1,
                                                                                                    2,
                                                                                                    3])])),
                                                                 ('ct2',
                                                                  ColumnTransformer(remainder='passthrough',
                                                                                    transformers=[('impute',
                                                                                                   SimpleImputer(),
                                                                                                   [0,
                                                                                                    1])])),
                                                                 ('ct3',
                                                                  ColumnTransformer(transformers=[('scaler',
                                                                                                   StandardScaler(),
                                                                                                   [0,
                                                                                                    1,
                                                                                                    2,
                                                                                                    3])]))])),
                                                ('cat_pipeline',
                                                 Pipeline(steps=[('cat',
                                                                  ColumnTransformer(transformers=[('ordinal_encode',
                                                                                                   OrdinalEncoder(),
                                                                                                   [4])]))]))])),
                ('selector', VarianceThreshold(threshold=0.1))])

In [26]:
transformed_X = full_pipeline.fit_transform(X)

In [27]:
transformed_X.shape

(748, 4)

In [28]:
from sklearn.feature_selection import RFE

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
estimator = LogisticRegression()
selector1 = RFE(estimator, n_features_to_select = 2)

In [31]:
X.shape

(748, 5)

In [33]:
X.head()

,V1,V2,V3,V4,V5
0,2.0,50.0,12500.0,98.0,NEGATIVE
1,0.0,13.0,3250.0,28.0,NEGATIVE
2,NaN,NaN,4000.0,35.0,NEGATIVE
3,NaN,20.0,5000.0,45.0,NEGATIVE
4,1.0,24.0,6000.0,77.0,NEGATIVE


In [34]:
y.head()

0    YES
1    YES
2    YES
3    YES
4     NO
Name: Target, dtype: object

In [35]:
encode = OrdinalEncoder()

In [39]:
y1 = y.to_numpy().reshape(-1,1)

In [41]:
y2 = encode.fit_transform(y1)

In [42]:
encode.categories_

[array(['NO', 'YES'], dtype=object)]

In [45]:
y2[:5]

array([[1.],
       [1.],
       [1.],
       [1.],
       [0.]])

In [54]:
select = selector1.fit(X_use, y2)
select.support_

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

array([ True, False,  True, False, False])

In [58]:
X_use[:5]

array([[-9.38169390e-01,  7.70986653e+00,  7.62334626e+00,
         2.61563344e+00,  0.00000000e+00],
       [-1.18627754e+00,  1.30454949e+00,  1.28273826e+00,
        -2.57880900e-01,  0.00000000e+00],
       [ 0.00000000e+00, -1.53758496e-16,  1.79684161e+00,
         2.94705348e-02,  0.00000000e+00],
       [ 0.00000000e+00,  2.51636623e+00,  2.48231275e+00,
         4.39972584e-01,  0.00000000e+00],
       [-1.06222347e+00,  3.20883294e+00,  3.16778388e+00,
         1.75357914e+00,  0.00000000e+00]])

In [55]:
from sklearn.feature_selection import SequentialFeatureSelector

In [56]:
sfsf = SequentialFeatureSelector(estimator, n_features_to_select = 2, direction = "forward")

In [59]:
sfs1 = sfsf.fit(X_use, y2)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [61]:
sfsf.support_

array([False,  True, False,  True, False])

In [64]:
sfsb = SequentialFeatureSelector(estimator, n_features_to_select = 2, direction = "backward")

In [65]:
sfsb.fit_transform(X_use, y2)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

array([[ 7.62334626,  2.61563344],
       [ 1.28273826, -0.2578809 ],
       [ 1.79684161,  0.02947053],
       ...,
       [-0.43093957,  1.13782607],
       [-0.77367514,  0.19367135],
       [-0.77367514,  1.54832812]])

In [66]:
sfsb.support_

array([False, False,  True,  True, False])